<a href="https://colab.research.google.com/github/biswa2005/ML-Learning/blob/main/Model_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
df['species'].value_counts()

,count
species,
setosa,50
versicolor,50
virginica,50


In [4]:
X = df.drop('species', axis=1)
y = df['species']

In [5]:
from sklearn.svm import SVC

model_svm = SVC(gamma = 'auto')

###GridSearchCV

In [6]:
from sklearn.model_selection import GridSearchCV

In [7]:
classifier = GridSearchCV((model_svm), {
    'C' : [1,10,20,30],
    'kernel' : ['rbf', 'linear'],
}, cv=5, return_train_score=False)

In [8]:
classifier.fit(X, y)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20, 30], 'kernel': ['rbf', 'linear']})

In [9]:
classifier.cv_results_

{'mean_fit_time': array([0.00376863, 0.00312595, 0.00327716, 0.00495186, 0.0084733 ,
        0.00777712, 0.01187725, 0.01256227]),
 'std_fit_time': array([4.66996561e-04, 1.15299225e-04, 8.17087429e-05, 2.08779022e-03,
        1.56855231e-03, 9.90125906e-04, 4.23164008e-03, 1.25990009e-03]),
 'mean_score_time': array([0.00283523, 0.00234165, 0.00254192, 0.00469856, 0.00614181,
        0.00542722, 0.008007  , 0.01025968]),
 'std_score_time': array([4.59749450e-04, 1.38138795e-04, 9.53453277e-05, 1.44363072e-03,
        2.83153001e-03, 1.77132323e-03, 2.23191563e-03, 1.46893532e-03]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20, 30, 30],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear',
                    'rbf', 'linear'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=np.str_('?'),
        

In [10]:
results = pd.DataFrame(classifier.cv_results_)

In [11]:
results[['param_C','param_kernel','mean_test_score']]


,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667
6,30,rbf,0.960000
7,30,linear,0.960000


###RandomizedSearchCV

In [12]:
from sklearn.model_selection import RandomizedSearchCV

In [13]:
Random_classifier = RandomizedSearchCV((model_svm), {
    'C' : [1,10,20,30],
    'kernel' : ['rbf', 'linear'],
}, n_iter=4, cv=5, return_train_score=False)

In [14]:
Random_classifier.fit(X, y)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=4,
                   param_distributions={'C': [1, 10, 20, 30],
                                        'kernel': ['rbf', 'linear']})

In [15]:
Random_classifier.cv_results_

{'mean_fit_time': array([0.00824928, 0.00858111, 0.00860653, 0.02119889]),
 'std_fit_time': array([0.00168737, 0.00181448, 0.00454201, 0.0054825 ]),
 'mean_score_time': array([0.00787306, 0.00520973, 0.00744367, 0.01347437]),
 'std_score_time': array([0.00157631, 0.00219844, 0.00252636, 0.00313771]),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'rbf'],
              mask=[False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'param_C': masked_array(data=[10, 30, 30, 1],
              mask=[False, False, False, False],
        fill_value=999999),
 'params': [{'kernel': 'rbf', 'C': 10},
  {'kernel': 'linear', 'C': 30},
  {'kernel': 'rbf', 'C': 30},
  {'kernel': 'rbf', 'C': 1}],
 'split0_test_score': array([0.96666667, 1.        , 0.96666667, 0.96666667]),
 'split1_test_score': array([1., 1., 1., 1.]),
 'split2_test_score': array([0.96666667, 0.9       , 0.9       , 0.96666667]),
 'split3_test_score': array([0.96666667, 0.9       , 0.933

In [16]:
results = pd.DataFrame(Random_classifier.cv_results_)

In [17]:
results[['param_C','param_kernel','mean_test_score']]


,param_C,param_kernel,mean_test_score
0,10,rbf,0.98
1,30,linear,0.96
2,30,rbf,0.96
3,1,rbf,0.98


###Ensemble Learning

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Stacking

In [21]:
from sklearn.ensemble import StackingClassifier

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [23]:
base_learners = [
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('svc', SVC(probability=True, kernel='rbf', random_state=42)),
    ('lr', LogisticRegression(max_iter=1000))
]

In [24]:
meta_learners = LogisticRegression(max_iter=1000)

In [25]:
stacking_clf = StackingClassifier(
  estimators=base_learners,
  final_estimator=meta_learners,
  cv=5
)

In [28]:
stacking_clf.fit(X_train, y_train)


StackingClassifier(cv=5,
                   estimators=[('dt', DecisionTreeClassifier(random_state=42)),
                               ('svc', SVC(probability=True, random_state=42)),
                               ('lr', LogisticRegression(max_iter=1000))],
                   final_estimator=LogisticRegression(max_iter=1000))

In [29]:
stacking_y_pred = stacking_clf.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score, classification_report


In [32]:
accuracy = accuracy_score(y_test, stacking_y_pred)
accuracy

1.0

Bagging

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
rf_clf = RandomForestClassifier(
    n_estimators = 100,
    random_state = 42
)

In [36]:
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [37]:
rf_y_pred = rf_clf.predict(X_test)

In [38]:
accuracy = accuracy_score(y_test, rf_y_pred)
accuracy

1.0

Boosting

In [39]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier


In [48]:
ada_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_model.fit(X_train,y_train)
ada_y_pred = ada_model.predict(X_test)
accuracy = accuracy_score(y_test, ada_y_pred)
accuracy

0.9333333333333333

In [49]:
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train,y_train)
gb_y_pred = gb_model.predict(X_test)
accuracy = accuracy_score(y_test, gb_y_pred)
accuracy

1.0

In [51]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, use_label_encoder=False, max_depth=3, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train,y_train_encoded)
xgb_y_pred_encoded = xgb_model.predict(X_test)
xgb_y_pred = le.inverse_transform(xgb_y_pred_encoded)
accuracy = accuracy_score(y_test, xgb_y_pred)
accuracy

1.0